In [1]:
#Importing dependencies
import config
import requests
import pandas as pd
import numpy as np
import datetime
from flask import Flask
from sqlalchemy import create_engine
import json
#Avoiding setting with copy warning
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing Redfin Housing Market State Data
raw_state_df = pd.read_csv("Resources/state_market_tracker.tsv000", sep='\t')
raw_state_df

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2019-10-01,2019-10-31,30,state,4,23,f,Oklahoma,NaN,Oklahoma,...,0.098560,0.209877,0.083561,0.053355,0.301282,-0.240768,-0.122115,South Region,NaN,2022-01-09 14:29:56
1,2021-07-01,2021-07-31,30,state,4,40,f,Vermont,NaN,Vermont,...,0.251473,0.133696,0.016402,-0.000131,0.424404,-0.064422,0.076156,Northeast Region,NaN,2022-01-09 14:29:56
2,2016-08-01,2016-08-31,30,state,4,10,f,New Hampshire,NaN,New Hampshire,...,0.089756,0.149826,0.009592,0.002687,0.206101,0.061385,0.016779,Northeast Region,NaN,2022-01-09 14:29:56
3,2013-04-01,2013-04-30,30,state,4,51,f,Mississippi,NaN,Mississippi,...,-0.008566,NaN,NaN,NaN,0.044777,-0.014869,-0.005006,South Region,NaN,2022-01-09 14:29:56
4,2019-12-01,2019-12-31,30,state,4,2,f,Missouri,NaN,Missouri,...,-0.017031,0.205339,-0.123381,-0.000410,0.252082,-0.116423,0.095007,Midwest Region,NaN,2022-01-09 14:29:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27073,2012-03-01,2012-03-31,30,state,4,6,f,Nebraska,NaN,Nebraska,...,0.021426,NaN,NaN,NaN,0.422780,0.042801,0.030384,Midwest Region,NaN,2022-01-09 14:29:56
27074,2016-07-01,2016-07-31,30,state,4,27,f,Idaho,NaN,Idaho,...,0.182390,NaN,NaN,NaN,0.534900,-0.117877,-0.004820,West Region,NaN,2022-01-09 14:29:56
27075,2020-11-01,2020-11-30,30,state,4,42,f,Virginia,NaN,Virginia,...,0.218939,0.126582,-0.048418,-0.006751,0.380567,-0.152381,0.194669,South Region,NaN,2022-01-09 14:29:56
27076,2021-08-01,2021-08-31,30,state,4,40,f,Vermont,NaN,Vermont,...,0.234272,0.158853,0.018284,0.022594,0.381659,-0.018233,0.098021,Northeast Region,NaN,2022-01-09 14:29:56


In [3]:
#Importing Redfin Housing Market State Data
raw_county_df = pd.read_csv("Resources/county_market_tracker.tsv000", sep='\t')
raw_county_df

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2017-04-01,2017-04-30,30,county,5,170,f,"Morgan County, AL",NaN,Alabama,...,-0.069595,NaN,NaN,NaN,0.020833,0.020833,0.001225,"Decatur, AL",19460.0,2022-01-09 14:29:56
1,2014-12-01,2014-12-31,30,county,5,1457,f,"Hennepin County, MN",NaN,Minnesota,...,-0.098136,0.097744,-0.121909,-0.197220,0.172414,0.034483,-0.033469,"Minneapolis, MN",33460.0,2022-01-09 14:29:56
2,2018-02-01,2018-02-28,30,county,5,2319,f,"Tulsa County, OK",NaN,Oklahoma,...,-0.024840,0.206004,-0.026139,0.009313,0.479924,0.068277,-0.042568,"Tulsa, OK",46140.0,2022-01-09 14:29:56
3,2017-08-01,2017-08-31,30,county,5,1448,f,"Crow Wing County, MN",NaN,Minnesota,...,NaN,0.200000,-0.050000,NaN,NaN,NaN,NaN,"Brainerd, MN",14660.0,2022-01-09 14:29:56
4,2019-05-01,2019-05-31,30,county,5,1586,f,"Tate County, MS",NaN,Mississippi,...,0.107895,NaN,NaN,NaN,0.000000,-0.111111,-0.100000,"Memphis, TN",32820.0,2022-01-09 14:29:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563117,2013-03-01,2013-03-31,30,county,5,2728,f,"Gonzales County, TX",NaN,Texas,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,Texas nonmetropolitan area,NaN,2022-01-09 14:29:56
563118,2013-09-01,2013-09-30,30,county,5,710,f,"Valley County, ID",NaN,Idaho,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,Idaho nonmetropolitan area,NaN,2022-01-09 14:29:56
563119,2016-06-01,2016-06-30,30,county,5,1261,f,"Livingston Parish, LA",NaN,Louisiana,...,-0.050180,0.189369,0.010930,0.028419,0.305785,-0.120571,0.020901,"Baton Rouge, LA",12940.0,2022-01-09 14:29:56
563120,2017-09-01,2017-09-30,30,county,5,2972,f,"Gloucester County, VA",NaN,Virginia,...,0.242424,NaN,NaN,NaN,0.000000,-0.111111,-0.166667,"Virginia Beach, VA",47260.0,2022-01-09 14:29:56


In [4]:
#Printing out column values to figure out what data we are interested in
print(raw_county_df.columns)

Index(['period_begin', 'period_end', 'period_duration', 'region_type',
       'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region',
       'city', 'state', 'state_code', 'property_type', 'property_type_id',
       'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy',
       'median_list_price', 'median_list_price_mom', 'median_list_price_yoy',
       'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf',
       'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold',
       'homes_sold_mom', 'homes_sold_yoy', 'pending_sales',
       'pending_sales_mom', 'pending_sales_yoy', 'new_listings',
       'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom',
       'inventory_yoy', 'months_of_supply', 'months_of_supply_mom',
       'months_of_supply_yoy', 'median_dom', 'median_dom_mom',
       'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom',
       'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom',
 

Property Type

Property type defined by RedFin
- All Residential: All properties defined as single-family, condominium, co-operative, townhouses, and multi-family (2-4 units) homes with a county record.
- Single Family Home (SFH): are homes built on a single lot, with no shared walls. Sometimes there’s a garage, attached or detached.
- Condominium (Condo): Usually a single unit within a larger building or community. Generally come with homeowners’ associations (HOAs), which require the residents to pay monthly or yearly dues.
- Cooperatives (Co-op): Usually a single unit within a larger building or community, but with a different way of holding a title to a shared building. You join a community and everyone in the community owns the building together.
- Townhouse: a hybrid between a condo and a single-family home. They are often multiple floors, with one or two shared walls, and some have a small yard space or rooftop deck. They’re generally larger than a condo, but smaller than a single-family home.
- Multifamily (2-4 units): They are essentially a home that has been turned into two or more units but the units cannot be purchased individually. There is one owner for the whole building.
- Land: Just land, no home of any type for sale.

Columns:

- Home sales: Total number of homes with a sale date during a given time period.
- Total active listings: The total number of listings that were active at any point during a given time period.
- Median sale price: The final home sale price covering all homes with a sale date during a given time period where 50% of the sales were above this price and 50% were below this price.
- Median sale price per square foot: The final home sale price divided by the total square feet of the property (not the lot) covering all homes with a sale date during a given time period where 50% of the sales were above this price per sqft and 50% were below this price per sqft.


In [5]:
#Checking for outliers
raw_county_df['property_type'].value_counts()

All Residential              179003
Single Family Residential    178662
Condo/Co-op                   74594
Multi-Family (2-4 Unit)       70143
Townhouse                     60720
Name: property_type, dtype: int64

In [11]:
raw_county_df['inventory']

0          439.0
1          133.0
2         2665.0
3            5.0
4           80.0
           ...  
563117      28.0
563118       7.0
563119     602.0
563120      79.0
563121     167.0
Name: inventory, Length: 563122, dtype: float64

In [13]:
county_df = raw_county_df[["period_end","region","state","property_type","inventory","homes_sold","median_sale_price","median_ppsf","parent_metro_region"]]
county_df

,period_end,region,state,property_type,inventory,homes_sold,median_sale_price,median_ppsf,parent_metro_region
0,2017-04-30,"Morgan County, AL",Alabama,All Residential,439.0,106.0,120500.0,75.845434,"Decatur, AL"
1,2014-12-31,"Hennepin County, MN",Minnesota,Multi-Family (2-4 Unit),133.0,57.0,203000.0,86.633333,"Minneapolis, MN"
2,2018-02-28,"Tulsa County, OK",Oklahoma,All Residential,2665.0,665.0,167500.0,89.765580,"Tulsa, OK"
3,2017-08-31,"Crow Wing County, MN",Minnesota,Condo/Co-op,5.0,3.0,125000.0,147.076613,"Brainerd, MN"
4,2019-05-31,"Tate County, MS",Mississippi,Single Family Residential,80.0,19.0,165000.0,85.000000,"Memphis, TN"
...,...,...,...,...,...,...,...,...,...
563117,2013-03-31,"Gonzales County, TX",Texas,All Residential,28.0,2.0,291500.0,83.404488,Texas nonmetropolitan area
563118,2013-09-30,"Valley County, ID",Idaho,Townhouse,7.0,1.0,221500.0,131.923764,Idaho nonmetropolitan area
563119,2016-06-30,"Livingston Parish, LA",Louisiana,All Residential,602.0,197.0,173990.0,108.063175,"Baton Rouge, LA"
563120,2017-09-30,"Gloucester County, VA",Virginia,Single Family Residential,79.0,9.0,170000.0,113.502506,"Virginia Beach, VA"
